# Data Modelisation

In [4]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LdaMulticore
import pickle 
import pyLDAvis

## 1) Data preparation

In [5]:
data = pd.read_csv("datasets/posts_clean.csv")
data.head(3)

,Id,Title,Body,Tags
0,415192,good way create simple python web service,I use python year I little experience python w...,<python><web-services>
1,415344,log implementation prefer,I implement log class c try decide I curious k...,<debugging><language-agnostic><logging>
2,414981,directly modify list element,I struct struct map public int size public map...,<c#><.net>


In [6]:
# remove < and > around Tags
data["Tags"] = data["Tags"].replace({"<" : " "}, regex=True)
data["Tags"] = data["Tags"].replace({">" : " "}, regex=True)

In [7]:
data.head()

,Id,Title,Body,Tags
0,415192,good way create simple python web service,I use python year I little experience python w...,python web-services
1,415344,log implementation prefer,I implement log class c try decide I curious k...,debugging language-agnostic logging
2,414981,directly modify list element,I struct struct map public int size public map...,c# .net
3,415580,regex name group java,understanding java regex package not support n...,java regex
4,415753,instance cache objective c,I want cache instance certain class class keep...,objective-c weak-references


In [8]:
docs = data["Body"].to_list()

In [9]:
# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [10]:
lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [11]:
# Compute bigrams.
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [12]:
# Remove rare and common tokens.
# Create a dictionary representation of the documents.
dct = Dictionary(docs)
dct.filter_extremes(no_below=20, no_above=0.5)

In [13]:
# Bag-of-words representation of the documents.
corpus = [dct.doc2bow(doc) for doc in docs]

In [14]:
print('Number of unique tokens: %d' % len(dct))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 24935
Number of documents: 164598


## 2) Topic Modelling

In [15]:
# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dct[0]  # This is only to "load" the dictionary.
id2word = dct.id2token

model = LdaModel(
    corpus=corpus,
    id2word = id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [16]:
coherence_model_lda = CoherenceModel(model=model, texts=docs, dictionary=dct, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5934874812843955


In [17]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = LdaMulticore(corpus=corpus,
                        id2word = id2word,
                        chunksize=chunksize,
                        alpha=a,
                        eta=b,
                        iterations=iterations,
                        num_topics=k,
                        passes=passes,
                        eval_every=eval_every)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dct, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [18]:
from gensim.utils import ClippedCorpus
import gensim
import tqdm

In [19]:
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dct, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 540/540 [38:30:52<00:00, 256.76s/it]


In [24]:
pd.DataFrame(model_results).sort_values("Coherence", ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
491,100% Corpus,9,0.61,0.31,0.640743
535,100% Corpus,10,asymmetric,0.01,0.636625
195,75% Corpus,8,0.91,0.01,0.629944
252,75% Corpus,10,0.61,0.61,0.629695
221,75% Corpus,9,0.61,0.31,0.626676
...,...,...,...,...,...
15,75% Corpus,2,0.91,0.01,0.440731
290,100% Corpus,2,symmetric,0.01,0.439272
280,100% Corpus,2,0.61,0.01,0.434665
279,100% Corpus,2,0.31,symmetric,0.434100


In [25]:
lda_model = LdaMulticore(corpus=corpus,
                        id2word=id2word,
                        num_topics=9, 
                        random_state=100,
                        chunksize=100,
                        passes=10,
                        alpha=0.61,
                        eta=0.31)

In [36]:
# Visualize the topics
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
lda_vis = gensimvis.prepare(lda_model, corpus, dct)

C:\Users\cecil\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [37]:
lda_vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.123740  0.005737       1        1  19.430566
4      0.134157 -0.054940       2        1  11.209940
0      0.000167 -0.313452       3        1  10.860619
1     -0.210625 -0.004800       4        1  10.688848
8      0.132191  0.013779       5        1  10.512200
5     -0.072090  0.153798       6        1  10.055112
3     -0.144666 -0.044692       7        1   9.881603
6     -0.044594  0.188848       8        1   8.937714
2      0.329200  0.055722       9        1   8.423398, topic_info=         Term          Freq         Total Category  logprob  loglift
308      file  86086.000000  86086.000000  Default  30.0000  30.0000
75        int  87626.000000  87626.000000  Default  29.0000  29.0000
93       java  65774.000000  65774.000000  Default  28.0000  28.0000
2063  android  58547.000000  58547.000000  Default  27.0000  27.0000
40      class  83029.000000  83029.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
104   support   5449.813544  12021.052043   Topic9  -5.5166   1.6831
40      class   7668.463473  83029.080005   Topic9  -5.1751   0.0921
2480     core   5411.687820  14741.942695   Topic9  -5.5236   1.4720
78        new   5604.653128  85183.439598   Topic9  -5.4886  -0.2471
462       run   5460.113033  51963.050104   Topic9  -5.5147   0.2211

[707 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
109       6  0.985485        abstract
109       9  0.013653        abstract
5655      6  0.996882  abstract_class
2061      1  0.000171        activity
2061      9  0.999457        activity
...     ...       ...             ...
31        1  0.165856            year
31        7  0.833825            year
7264      1  0.989281        year_ago
2312      7  0.996558            yyyy
2312      8  0.002192            yyyy

[2247 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 1, 2, 9, 6, 4, 7, 3])